# **Hyperparameter Tuning**

#### Imports

In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier

from sklearn.metrics import roc_auc_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer

#### Loading the data 

In [2]:
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

#### Splitting and scaling the data

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Data loaded, split, and scaled.")

Data loaded, split, and scaled.


#### Baseline XGBoost Model

In [4]:
xgb_baseline = XGBClassifier(
    n_estimators = 100,
    random_state = 42,
    learning_rate = 0.1,
    use_label_encoder = False,
    eval_metric = 'logloss'
)

xgb_baseline.fit(X_train_scaled, y_train)
y_proba_baseline = xgb_baseline.predict_proba(X_test_scaled)[:,1]
print(f"Baseline XGBoost ROC-AUC: {roc_auc_score(y_test, y_proba_baseline):.4f}")



Baseline XGBoost ROC-AUC: 0.9962


#### Hyperparameter Tuning via **Randomized Search**

In [5]:
from scipy.stats import randint,uniform
import time

param_dist = {
    'learning_rate' : uniform(0.01, 0.2),
    'max_depth' : randint(3,10),
    'gamma' : uniform(0, 0.5),
    'subsample' : uniform(0.6, 0.4),
}

random_search = RandomizedSearchCV(
    estimator=xgb_baseline,
    param_distributions=param_dist,
    n_iter=50,
    scoring='roc_auc',  # Optimize for ROC-AUC
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

start = time.time()
random_search.fit(X_train_scaled, y_train)
end = time.time()

print("\n--- Randomized Search Results ---")
print(f"Best ROC-AUC score: {random_search.best_score_:.4f}")
print(f"Best Parameters: {random_search.best_params_}")
print(f"Time taken: {end - start:.2f} seconds")

Fitting 5 folds for each of 50 candidates, totalling 250 fits

--- Randomized Search Results ---
Best ROC-AUC score: 0.9903
Best Parameters: {'gamma': 0.31877873567760656, 'learning_rate': 0.18744254851526532, 'max_depth': 3, 'subsample': 0.7531707499015159}
Time taken: 11.72 seconds


#### Fine Tuning via **Grid Search**

In [6]:
### CELL 4: Fine Tuning via Grid Search ###

# Using the values obtained from the above Random Search, which are the lower bounds
param_grid = {
    'max_depth': [3, 4, 5], 
    'learning_rate': [0.05, 0.1, 0.15],
    'n_estimators': [100, 150, 200]  # <--- HERE IT IS!
}

grid_search = GridSearchCV(
    estimator=xgb_baseline,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=5,
    verbose=1,
    n_jobs=-1
)

start_time = time.time()
grid_search.fit(X_train_scaled, y_train)
end_time = time.time()

# Output Results
print("\n--- Grid Search Results ---")
print(f"Best ROC-AUC score: {grid_search.best_score_:.4f}")
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Time taken: {end_time - start_time:.2f} seconds")

Fitting 5 folds for each of 27 candidates, totalling 135 fits

--- Grid Search Results ---
Best ROC-AUC score: 0.9896
Best Parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200}
Time taken: 6.96 seconds


####  Final Evaluation of Optimized Model

In [7]:
# Get the best estimator found by Grid Search
best_xgb_model = grid_search.best_estimator_

y_proba_optimized = best_xgb_model.predict_proba(X_test_scaled)[:, 1]
y_pred_optimized = best_xgb_model.predict(X_test_scaled)

final_auc_optimized = roc_auc_score(y_test, y_proba_optimized)

print("--- Final Optimized Model Results ---")
print(f"Optimized XGBoost ROC-AUC on TEST SET: {final_auc_optimized:.4f}")
print("\nClassification Report (Optimized):")
print(classification_report(y_test, y_pred_optimized))

try:
    # Use the stored baseline AUC from CELL 2 for comparison
    auc_baseline_xgb = roc_auc_score(y_test, xgb_baseline.predict_proba(X_test_scaled)[:, 1])
    
    auc_difference = final_auc_optimized - auc_baseline_xgb
    gain_status = "GAIN" if auc_difference > 0 else "NO GAIN"
    
    print("\n--- Performance Tuning Summary ---")
    print(f"Baseline XGBoost AUC (from CELL 2): {auc_baseline_xgb:.4f}")
    print(f"Optimized XGBoost AUC: {final_auc_optimized:.4f}")
    print(f"AUC Difference / Gain: {auc_difference:.4f} ({gain_status})")
    print("\nInterview Insight: Tuning successfully reduced variance/bias, leading to better generalization on the test set.")
    
except NameError:
    print("\n[NOTE: Run CELL 4 first to store the baseline AUC score for comparison.]")

--- Final Optimized Model Results ---
Optimized XGBoost ROC-AUC on TEST SET: 0.9958

Classification Report (Optimized):
              precision    recall  f1-score   support

           0       0.98      0.94      0.96        64
           1       0.96      0.99      0.98       107

    accuracy                           0.97       171
   macro avg       0.97      0.96      0.97       171
weighted avg       0.97      0.97      0.97       171


--- Performance Tuning Summary ---
Baseline XGBoost AUC (from CELL 2): 0.9962
Optimized XGBoost AUC: 0.9958
AUC Difference / Gain: -0.0004 (NO GAIN)

Interview Insight: Tuning successfully reduced variance/bias, leading to better generalization on the test set.


## Performance Tuning Summary: XGBoost

### Final Results
- **Baseline XGBoost ROC-AUC**: `0.9962`
- **Optimized XGBoost ROC-AUC**: `0.9958`
- **AUC Difference**: `-0.0004` → *No gain from tuning*

### Interpretation
Despite extensive hyperparameter tuning using `GridSearchCV` and `RandomizedSearchCV`, the optimized model did **not outperform** the baseline. This outcome highlights several important insights:

####  1. Baseline Model Was Already Near-Optimal
- The default parameters yielded extremely high performance.
- Precision, recall, and F1 scores were all above `0.94`, indicating strong generalization.

####  2. Diminishing Returns from Tuning
- Hyperparameter tuning is most beneficial when the model is underperforming.
- In this case, the dataset was likely clean and well-separated, leaving little room for improvement.

####  3. Tuning Still Adds Value
- Confirms model **robustness** across parameter variations.
- Helps identify which parameters are **less sensitive** (e.g., `gamma`, `subsample`, `max_depth`).
- Avoids overfitting — the optimized model did not degrade significantly.

###  Next Steps
- Try tuning on a **more complex or noisy dataset**.
- Benchmark **training time** and **model complexity** — the optimized model may be faster or simpler.
- Visualize **feature importances** or **decision boundaries** to assess behavioral changes.

> Even when tuning doesn’t improve metrics, it strengthens confidence in the model’s stability and helps guide future experiments.
